In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import fsolve
from scipy.optimize import brentq

$$\begin{aligned}
P_0 &= 100mio P_swap_0 + 1mio AAPL_0 + 1mio MSFT_0 + 1mio F + 1mio BAC_0     \\
P_1 &= 100mio P_swap_1 + 1mio AAPL_1 + 1mio MSFT_1 + 1mio F_1 + 1mio BAC_1   \\

\Delta_1 L &= P_1 - P_0  \\
&=  \left[ 1e8 S_{swap}(1) + 1e6 S_{aapl}(1)  + 1e6 S_{msft}(1)  + 1e6 S_{f}(1)  + 1e6 S_{bac}(1) \right] - \left[ 1e8 S_{swap}(0) + 1e6 S_{aapl}(0)  + 1e6 S_{msft}(0)  + 1e6 S_{f}(0)  + 1e6 S_{bac}(0)  \right] \\
    &= \left[ 1e8 S_{swap}(1) - 1e8 S_{swap}(0) \right] + \left[ 1e6 S_{aapl}(1) - 1e6 S_{aapl}(0) \right]  + \left[ 1e6 S_{msft}(1) - 1e6 S_{msft}(0) \right] + \left[ 1e6 S_{f}(1) - 1e6 S_{f}(0) \right] + \left[ 1e6 S_{bac}(1) - 1e6 S_{bac}(0) \right] \\
    &= 1e6 S_A(0)  \left[ \prod_{i=0}^{9} (1 + R_A^i) \prod_{i=0}^{9} (1 + R_{fx}^i) - 1 \right] + 200000 S_D(0) \left[ \prod_{i=0}^{9} (1 + R_D^i) - 1 \right]                     \\

\Delta_1 L (\mu , \sigma^2) \\
\\
\mu \approx &100mio \mathbb{E}[\Delta_{swap}] + 1mio \mathbb{E}&[\Delta_{aapl} + \Delta_{msft}  + \Delta_{f}  + \Delta_{bac}]        \\

\sigma^2 \approx &\text{Var}(100mio\Delta_{swap}) + \text{Var}(1mio [\Delta_{aapl} + \Delta_{msft}  + \Delta_{f}  + \Delta_{bac}])
\end{aligned}$$

## for swap sensitivity change 
get partial differenctial by changing one tenor by 1 bp then mark the change in PV \
$$
\Delta PV = N \left[ \sum_{i=1}^T PV01_i \times \Delta R_i  \right]
$$
where N is the notional of swap

**correlation between SOFR and stocks matter**

## for swap full eval
use new OIS curve to price swap at 4.2%

In [2]:
aapl = pd.read_excel("./data/hist_data.xlsm", sheet_name = 'AAPL')
msft = pd.read_excel("./data/hist_data.xlsm", sheet_name = 'MSFT')
f = pd.read_excel("./data/hist_data.xlsm", sheet_name = 'F')
bac = pd.read_excel("./data/hist_data.xlsm", sheet_name = 'BAC')
sofr_curve = pd.read_excel("./data/hist_data.xlsm", sheet_name = 'SofrCurve')

# Extract "T" as a separate array
T_array = sofr_curve["T"].values
df_transposed = sofr_curve.set_index("T").T

# Rename index to "Date"
df_transposed.index.name = "Date"
df_transposed.reset_index(inplace=True)
col_names = ['Date'] + list(df_transposed.loc[0][1:].values)
df_transposed.columns = col_names
sofr_curve = df_transposed[1:]
sofr_curve['Date'] = pd.to_datetime(sofr_curve['Date'])
sofr_curve.reset_index(inplace=True,drop=True)

/var/folders/h2/r7qn2m9n1zb6y_0q191gdqth0000gn/T/ipykernel_3740/1798135338.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sofr_curve['Date'] = pd.to_datetime(sofr_curve['Date'])


In [3]:
df_stocks = pd.concat([aapl,
                        msft['Adj Close'],
                        f['Adj Close'],
                        bac['Adj Close']],axis=1)
df_stocks.columns = ['Date', 'aapl', 'msft', 'f', 'bac']

In [ ]:
df_stocks.merge(sofr_curve,
           on=['Date'],
           how='outer').ffill() # TODO outer join because dates dont match

/var/folders/h2/r7qn2m9n1zb6y_0q191gdqth0000gn/T/ipykernel_3740/718887081.py:3: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  how='outer').ffill() # TODO outer join because dates dont match


,Date,Adj Close,1D,1M,2M,3M,6M,9M,1Y,2Y,...,15Y,16Y,17Y,18Y,19Y,20Y,25Y,30Y,35Y,40Y
0,2022-10-31,152.041122,0.039191,0.038721,0.038670,0.040536,0.044577,0.046004,0.046449,0.044583,...,0.037151,0.037057,0.036907,0.036698,0.036433,0.036111,0.034091,0.032350,0.030552,0.028708
1,2022-11-01,149.373917,0.039604,0.039023,0.038886,0.040725,0.044849,0.046448,0.046970,0.045022,...,0.036802,0.036682,0.036511,0.036287,0.036010,0.035678,0.033645,0.031979,0.030238,0.028478
2,2022-11-02,143.801514,0.039948,0.039286,0.039100,0.040852,0.044884,0.046580,0.047203,0.045496,...,0.036855,0.036701,0.036498,0.036248,0.035953,0.035613,0.033627,0.031936,0.030292,0.028608
3,2022-11-03,137.703613,0.040389,0.039585,0.039350,0.041154,0.045281,0.047107,0.047894,0.046594,...,0.037221,0.037069,0.036886,0.036657,0.036372,0.036022,0.033811,0.032134,0.030407,0.028655
4,2022-11-04,137.435455,0.045965,0.042343,0.038795,0.040611,0.045212,0.046752,0.047500,0.046097,...,0.037687,0.037557,0.037380,0.037152,0.036870,0.036534,0.034424,0.032558,0.030723,0.028933
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
248,2023-10-24,172.991058,0.053105,0.053084,0.053202,0.053424,0.053690,0.053293,0.052503,0.048399,...,0.044093,0.044056,0.043972,0.043841,0.043663,0.043437,0.041874,0.040238,0.038438,0.036493
249,2023-10-25,170.657135,0.052981,0.053057,0.053259,0.053490,0.053749,0.053393,0.052653,0.048791,...,0.045289,0.045268,0.045198,0.045078,0.044906,0.044683,0.043076,0.041358,0.039564,0.037641
250,2023-10-26,166.458023,0.053047,0.053089,0.053218,0.053414,0.053567,0.053089,0.052243,0.048044,...,0.044269,0.044244,0.044171,0.044049,0.043882,0.043668,0.042182,0.040589,0.038760,0.036764
251,2023-10-27,167.784576,0.052989,0.053040,0.053185,0.053368,0.053486,0.052991,0.052115,0.047758,...,0.044460,0.044461,0.044411,0.044309,0.044157,0.043955,0.042508,0.040985,0.039167,0.037148


In [5]:
sofr_curve[:20]

,Date,1D,1M,2M,3M,6M,9M,1Y,2Y,3Y,...,15Y,16Y,17Y,18Y,19Y,20Y,25Y,30Y,35Y,40Y
0,2022-10-31,0.039191,0.038721,0.03867,0.040536,0.044577,0.046004,0.046449,0.044583,0.042002,...,0.037151,0.037057,0.036907,0.036698,0.036433,0.036111,0.034091,0.03235,0.030552,0.028708
1,2022-11-01,0.039604,0.039023,0.038886,0.040725,0.044849,0.046448,0.04697,0.045022,0.042344,...,0.036802,0.036682,0.036511,0.036287,0.03601,0.035678,0.033645,0.031979,0.030238,0.028478
2,2022-11-02,0.039948,0.039286,0.0391,0.040852,0.044884,0.04658,0.047203,0.045496,0.042749,...,0.036855,0.036701,0.036498,0.036248,0.035953,0.035613,0.033627,0.031936,0.030292,0.028608
3,2022-11-03,0.040389,0.039585,0.03935,0.041154,0.045281,0.047107,0.047894,0.046594,0.043833,...,0.037221,0.037069,0.036886,0.036657,0.036372,0.036022,0.033811,0.032134,0.030407,0.028655
4,2022-11-04,0.045965,0.042343,0.038795,0.040611,0.045212,0.046752,0.0475,0.046097,0.043385,...,0.037687,0.037557,0.03738,0.037152,0.03687,0.036534,0.034424,0.032558,0.030723,0.028933
5,2022-11-07,0.040224,0.039258,0.03946,0.041424,0.045455,0.047189,0.047929,0.046633,0.043884,...,0.038118,0.038021,0.037868,0.037655,0.037383,0.037047,0.034922,0.033135,0.0313,0.029495
6,2022-11-08,0.040339,0.039211,0.03942,0.041406,0.045334,0.046941,0.047589,0.046046,0.043209,...,0.037361,0.037271,0.037122,0.036917,0.036657,0.036344,0.03439,0.032672,0.030902,0.029141
7,2022-11-09,0.040373,0.039193,0.039483,0.041445,0.045299,0.046883,0.047479,0.045585,0.042645,...,0.037715,0.037659,0.037544,0.037371,0.037138,0.036845,0.034969,0.033394,0.031539,0.029678
8,2022-11-10,0.052075,0.043694,0.038786,0.040867,0.044951,0.045754,0.045989,0.042912,0.039631,...,0.035006,0.035,0.03494,0.034825,0.034656,0.034433,0.032875,0.031361,0.029648,0.027861
9,2022-11-14,0.040975,0.039174,0.039732,0.041627,0.044918,0.046142,0.046469,0.043394,0.039989,...,0.035086,0.035074,0.035014,0.034902,0.034735,0.03451,0.032961,0.031597,0.029815,0.027985


In [6]:
aapl.head(20)

,Date,Adj Close
0,2022-10-31,152.041122
1,2022-11-01,149.373917
2,2022-11-02,143.801514
3,2022-11-03,137.703613
4,2022-11-04,137.435455
5,2022-11-07,137.971741
6,2022-11-08,138.547806
7,2022-11-09,133.949402
8,2022-11-10,145.867508
9,2022-11-11,148.678192


In [7]:
df_sofr.pct_change()

NameError: name 'df_sofr' is not defined

In [ ]:
df_sofr.head()

,1D,1M,2M,3M,6M,9M,1Y,2Y,3Y,4Y,...,15Y,16Y,17Y,18Y,19Y,20Y,25Y,30Y,35Y,40Y
0,0.039191,0.038721,0.03867,0.040536,0.044577,0.046004,0.046449,0.044583,0.042002,0.040318,...,0.037151,0.037057,0.036907,0.036698,0.036433,0.036111,0.034091,0.03235,0.030552,0.028708
1,0.039604,0.039023,0.038886,0.040725,0.044849,0.046448,0.04697,0.045022,0.042344,0.040614,...,0.036802,0.036682,0.036511,0.036287,0.03601,0.035678,0.033645,0.031979,0.030238,0.028478
2,0.039948,0.039286,0.0391,0.040852,0.044884,0.04658,0.047203,0.045496,0.042749,0.040868,...,0.036855,0.036701,0.036498,0.036248,0.035953,0.035613,0.033627,0.031936,0.030292,0.028608
3,0.040389,0.039585,0.03935,0.041154,0.045281,0.047107,0.047894,0.046594,0.043833,0.041825,...,0.037221,0.037069,0.036886,0.036657,0.036372,0.036022,0.033811,0.032134,0.030407,0.028655
4,0.045965,0.042343,0.038795,0.040611,0.045212,0.046752,0.0475,0.046097,0.043385,0.041503,...,0.037687,0.037557,0.03738,0.037152,0.03687,0.036534,0.034424,0.032558,0.030723,0.028933


In [ ]:
df_stocks.cov()

,aapl,msft,f,bac
aapl,351.380323,724.105442,9.918896,-35.916317
msft,724.105442,1622.654311,16.021257,-90.258729
f,9.918896,16.021257,0.958969,0.264916
bac,-35.916317,-90.258729,0.264916,9.617016


In [ ]:
df_sofr.cov().head()

,1D,1M,2M,3M,6M,9M,1Y,2Y,3Y,4Y,...,15Y,16Y,17Y,18Y,19Y,20Y,25Y,30Y,35Y,40Y
1D,0.000022,0.000022,0.000019,0.000016,0.000012,0.000010,0.000008,0.000006,0.000007,0.000008,...,0.000009,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010
1M,0.000022,0.000023,0.000021,0.000018,0.000013,0.000011,0.000008,0.000006,0.000007,0.000007,...,0.000009,0.000009,0.000009,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010
2M,0.000019,0.000021,0.000019,0.000017,0.000012,0.000010,0.000008,0.000006,0.000006,0.000007,...,0.000008,0.000008,0.000008,0.000008,0.000008,0.000009,0.000009,0.000009,0.000009,0.000009
3M,0.000016,0.000018,0.000017,0.000014,0.000011,0.000009,0.000007,0.000006,0.000006,0.000006,...,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008
6M,0.000012,0.000013,0.000012,0.000011,0.000009,0.000008,0.000007,0.000007,0.000007,0.000007,...,0.000007,0.000007,0.000007,0.000007,0.000007,0.000007,0.000007,0.000007,0.000007,0.000007
